In [2]:
# Import libraries
import math
import numpy as np
import scipy.stats
import itertools


## 1.) Full Loop, no pre-allocation 


In [2]:
# Function to compute all confidence intervals
def ciallf(x, z, n, a1, a2):
  
  # Wald CI
    c0  = x/n   
    c1  = z*(math.sqrt(c0*(1 - c0)/n))

  # Wilson CI
    b0    = (1/(1 +  z**2/n))*(c0 + (z**2)/(2*n))
    b1    = (z/(1 + (z**2/n)))*math.sqrt((c0*(1 - c0)/n) + z**2/(4*n**2)  )

  # Lower Clopper Pearson
    if x == 0:
      cplow = 0
    else:
      cplow = scipy.stats.beta.ppf(a1, x, n - x + 1)
      
  # Upper Clopper Pearson
    if x == n:
      cpup = 1
    else:
      cpup = scipy.stats.beta.ppf(a2, x + 1, n - x) 
      
  # Return values of function    
    return np.array([(c0 - c1), (c0 + c1), 
                     (b0 - b1), (b0 + b1),
                     cplow, cpup]) 
      
# Set parameters   
n     = 100
N     = 10_000_000
p     = 0.1
alpha = 0.05
z     = scipy.stats.norm.ppf(1 - 0.05/2)
x     = np.random.binomial(n, p, N) 
a1    = alpha/2
a2    = (1 - alpha/2)

# Pre allocate result vector
vec_covered = np.empty((N, 3), dtype = bool)

for i, xval in enumerate(x):
  results           = ciallf(xval, z, n, a1, a2) 
  vec_covered[i, 0] = results[0] < p < results[1]
  vec_covered[i, 1] = results[2] < p < results[3]
  vec_covered[i, 2] = results[4] < p < results[5]
  
# Cover rates
print(sum(vec_covered[:, 0])/N) 
print(sum(vec_covered[:, 1])/N) 
print(sum(vec_covered[:, 2])/N) 

0.9326073
0.9364108
0.9557936


## 2.) Half vectorized

In [6]:
# Function to compute all confidence intervals
def ciallf(x, z, n, cpup, cplow):
  
  # Wald CI
    c0  = x/n   
    c1  = z*(math.sqrt(c0*(1 - c0)/n))

  # Wilson CI
    b0    = (1/(1 +  z**2/n))*(c0 + (z**2)/(2*n))
    b1    = (z/(1 + (z**2/n)))*math.sqrt((c0*(1 - c0)/n) + z**2/(4*n**2)  )

  # Clopper Pearson CI
    cpup  = cpup[x]
    cplow = cplow[x]
      
  # Return values of function    
    return np.array([(c0 - c1), (c0 + c1), 
                     (b0 - b1), (b0 + b1),
                     cplow, cpup]) 
      
# Set parameters   
n     = 100
N     = 10_000_000
p     = 0.1
alpha = 0.05
z     = scipy.stats.norm.ppf(1 - 0.05/2)
x     = np.random.binomial(n, p, N) 
a1    = alpha/2
a2    = (1 - alpha/2)

# Pre-define Clopper-Pearson confidence intervals
cpup = np.array([scipy.stats.beta.ppf(a2, x + 1, n - x) for x in range(101)])
cpup = np.nan_to_num(cpup, nan = 1.0)

cplow = np.array([scipy.stats.beta.ppf(a1, x, n - x + 1) for x in range(101)])
cplow = np.nan_to_num(cplow, nan = 0.0)

# Pre allocate result vector
vec_covered = np.empty((N, 3), dtype = bool)

for i, xval in enumerate(x):
  results           = ciallf(xval, z, n, cpup, cplow) 
  vec_covered[i, 0] = results[0] < p < results[1]
  vec_covered[i, 1] = results[2] < p < results[3]
  vec_covered[i, 2] = results[4] < p < results[5]
  
# Cover rates
print(sum(vec_covered[:, 0])/N) 
print(sum(vec_covered[:, 1])/N) 
print(sum(vec_covered[:, 2])/N) 